# Pair Distribution Function: NaCl, XRD

This example demonstrates a pair distribution function (PDF) analysis of
NaCl, based on data collected from an X-ray powder diffraction experiment.

The dataset is taken from:
https://github.com/diffpy/add2019-diffpy-cmi/tree/master

## Import Library

In [ ]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        print('Installing the easydiffraction library...')
        !pip install 'easydiffraction[visualization]'

In [ ]:
import easydiffraction as ed

## Create Project

In [ ]:
project = ed.Project()

## Set Plotting Engine

In [ ]:
project.plotter.engine = 'plotly'
project.plotter.x_min = 2.0
project.plotter.x_max = 30.0

## Add Sample Model

In [ ]:
project.sample_models.add(name='nacl')

In [ ]:
project.sample_models['nacl'].space_group.name_h_m = 'F m -3 m'
project.sample_models['nacl'].space_group.it_coordinate_system_code = '1'
project.sample_models['nacl'].cell.length_a = 5.62
project.sample_models['nacl'].atom_sites.add(
    label='Na', type_symbol='Na', fract_x=0, fract_y=0, fract_z=0, wyckoff_letter='a', b_iso=1.0
)
project.sample_models['nacl'].atom_sites.add(
    label='Cl', type_symbol='Cl', fract_x=0.5, fract_y=0.5, fract_z=0.5, wyckoff_letter='b', b_iso=1.0
)

## Add Experiment

In [ ]:
ed.download_from_repository('NaCl.gr', destination='data')

In [ ]:
project.experiments.add(
    name='xray_pdf',
    sample_form='powder',
    beam_mode='constant wavelength',
    radiation_probe='xray',
    scattering_type='total',
    data_path='data/NaCl.gr',
)

In [ ]:
project.experiments['xray_pdf'].show_supported_peak_profile_types()

In [ ]:
project.experiments['xray_pdf'].show_current_peak_profile_type()

In [ ]:
project.experiments['xray_pdf'].peak_profile_type = 'gaussian-damped-sinc'

In [ ]:
project.experiments['xray_pdf'].peak.damp_q = 0.03
project.experiments['xray_pdf'].peak.broad_q = 0
project.experiments['xray_pdf'].peak.cutoff_q = 21
project.experiments['xray_pdf'].peak.sharp_delta_1 = 0
project.experiments['xray_pdf'].peak.sharp_delta_2 = 5
project.experiments['xray_pdf'].peak.damp_particle_diameter = 0

In [ ]:
project.experiments['xray_pdf'].linked_phases.add(id='nacl', scale=0.5)

## Select Fitting Parameters

In [ ]:
project.sample_models['nacl'].cell.length_a.free = True
project.sample_models['nacl'].atom_sites['Na'].b_iso.free = True
project.sample_models['nacl'].atom_sites['Cl'].b_iso.free = True

In [ ]:
project.experiments['xray_pdf'].linked_phases['nacl'].scale.free = True
project.experiments['xray_pdf'].peak.damp_q.free = True
project.experiments['xray_pdf'].peak.sharp_delta_2.free = True

## Run Fitting

In [ ]:
project.analysis.current_calculator = 'pdffit'
project.analysis.fit()

## Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='xray_pdf')